<a href="https://colab.research.google.com/github/shuaicongbaobao/Colab-140/blob/main/The_real_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gc
import os
import random
import warnings
import numpy as np
import pandas as pd
from IPython.display import display

!pip install timm
import timm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import CosineAnnealingLR

from google.colab import drive
drive.mount('/content/drive')

from scipy import signal


warnings.filterwarnings('ignore', category=Warning)
gc.collect()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylin

0

In [2]:
df = pd.read_csv('/content/drive/MyDrive/hms-harmful-brain-activity-classification/train.csv')
TARGETS = df.columns[-6:]
print('Train shape:', df.shape )
print('Targets', list(TARGETS))
df.head()

Train shape: (106800, 15)
Targets ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']


,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0


In [3]:
train = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_id':'first','spectrogram_label_offset_seconds':'min'})
train.columns = ['spec_id','min']

tmp = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_label_offset_seconds':'max'})
train['max'] = tmp

tmp = df.groupby('eeg_id')[['patient_id']].agg('first')
train['patient_id'] = tmp

tmp = df.groupby('eeg_id')[TARGETS].agg('sum')
for t in TARGETS:
    train[t] = tmp[t].values

y_data = train[TARGETS].values
y_data = y_data / y_data.sum(axis=1,keepdims=True)
train[TARGETS] = y_data

tmp = df.groupby('eeg_id')[['expert_consensus']].agg('first')
train['target'] = tmp

train = train.reset_index()
print('Train non-overlapp eeg_id shape:', train.shape )
train.head()

Train non-overlapp eeg_id shape: (17089, 12)


,eeg_id,spec_id,min,max,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,target
0,568657,789577333,0.0,16.0,20654,0.0,0.000000,0.25,0.000000,0.166667,0.583333,Other
1,582999,1552638400,0.0,38.0,20230,0.0,0.857143,0.00,0.071429,0.000000,0.071429,LPD
2,642382,14960202,1008.0,1032.0,5955,0.0,0.000000,0.00,0.000000,0.000000,1.000000,Other
3,751790,618728447,908.0,908.0,38549,0.0,0.000000,1.00,0.000000,0.000000,0.000000,GPD
4,778705,52296320,0.0,0.0,40955,0.0,0.000000,0.00,0.000000,0.000000,1.000000,Other


In [4]:
%%time
READ_SPEC_FILES = False

# READ ALL SPECTROGRAMS
PATH = '/content/drive/MyDrive/hms-harmful-brain-activity-classification/train_spectrograms/'
files = os.listdir(PATH)
print(f'There are {len(files)} spectrogram parquets')

if READ_SPEC_FILES:
    spectrograms = {}
    for i,f in enumerate(files):
        if i%100==0: print(i,', ',end='')
        tmp = pd.read_parquet(f'{PATH}{f}')
        name = int(f.split('.')[0])
        spectrograms[name] = tmp.iloc[:,1:].values
else:
    spectrograms = np.load('/content/drive/MyDrive/brain-spectrograms/specs.npy',allow_pickle=True).item()
print('Train spectrograms read!')

There are 11166 spectrogram parquets
Train spectrograms read!
CPU times: user 839 ms, sys: 6.64 s, total: 7.47 s
Wall time: 2min 23s


In [5]:
%%time
READ_EEG_SPEC_FILES = False

if READ_EEG_SPEC_FILES:
    all_eegs = {}
    for i,e in enumerate(train.eeg_id.values):
        if i%100==0: print(i,', ',end='')
        x = np.load(f'/content/drive/MyDrive/brain-spectrograms/EEG_Spectrograms/{e}.npy')
        all_eegs[e] = x
else:
    all_eegs = np.load('/content/drive/MyDrive/archive/eeg_specs.npy',allow_pickle=True).item()
print('EEG generated spectrograms read!')

EEG generated spectrograms read!
CPU times: user 1.02 s, sys: 9.09 s, total: 10.1 s
Wall time: 1min 44s


In [66]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import albumentations as albu

TARGETS = {'Seizure': 0, 'LPD': 1, 'GPD': 2, 'LRDA': 3, 'GRDA': 4, 'Other': 5}
TARGETS2 = {x: y for y, x in TARGETS.items()}

class DataGenerator(Dataset):
    'Generates data for PyTorch'
    def __init__(self, data, batch_size=32, shuffle=False, augment=False, mode='train', specs=spectrograms, eeg_specs=all_eegs):
        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augment = augment
        self.mode = mode
        self.specs = specs
        self.eeg_specs = eeg_specs
        self.indexes = np.arange(len(self.data))
        columns = data.columns.tolist()
        self.target_indexes = [columns.index(t.lower() + "_vote") for t in TARGETS]
        if self.shuffle:
            np.random.shuffle(self.indexes)
    def __len__(self):
        return int(np.floor(len(self.data) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__data_generation(indexes)
        if self.augment:
            X = self.__augment_batch(X)
        return torch.from_numpy(X), torch.from_numpy(y)

    def __data_generation(self, indexes):
        X = np.zeros((len(indexes), 128, 256, 8), dtype='float32')
        y = np.zeros((len(indexes), 6), dtype='float32')

        for j, i in enumerate(indexes):
            row = self.data.iloc[i]
            if self.mode == 'test':
                r = 0
            else:
                r = int((row['min'] + row['max']) // 4)

            for k in range(4):
                img = self.specs[row.spec_id][r:r + 300, k * 100:(k + 1) * 100].T
                img = np.clip(img, np.exp(-4), np.exp(8))
                img = np.log(img)

                ep = 1e-6
                m = np.nanmean(img.flatten())
                s = np.nanstd(img.flatten())
                img = (img - m) / (s + ep)
                img = np.nan_to_num(img, nan=0.0)

                X[j, 14:-14, :, k] = img[:, 22:-22] / 2.0

            img = self.eeg_specs[row.eeg_id]
            X[j, :, :, 4:] = img
            if self.mode != 'test':
                y[j] = row[self.target_indexes]

        return X, y

    def __random_transform(self, img):
        composition = albu.Compose([
            albu.HorizontalFlip(p=0.5),
            # You can add more transformations here
        ])
        img = composition(image=img)['image']
        return img

    def __augment_batch(self, img_batch):
        for i in range(img_batch.shape[0]):
            img_batch[i] = self.__random_transform(img_batch[i])
        return img_batch


In [32]:
print(train.columns)
print(train.iloc[train_index].columns)

Index(['eeg_id', 'spec_id', 'min', 'max', 'patient_id', 'seizure_vote',
       'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote',
       'target'],
      dtype='object')
Index(['eeg_id', 'spec_id', 'min', 'max', 'patient_id', 'seizure_vote',
       'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote',
       'target'],
      dtype='object')


In [67]:
import torch
import torch.nn as nn

class DeepSpeech2(nn.Module):
    def __init__(self, input_shape=(128, 256, 8), num_classes=6):
        super(DeepSpeech2, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(input_shape[2], 128, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=(7, 7), stride=2, padding=(3, 3)),
            nn.ReLU()
        )
        self.gru = nn.GRU(input_size=128, hidden_size=64, batch_first=True)
        self.fc = nn.Linear(64, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = torch.permute(x, (0, 2,3, 1))
        x = x.reshape(x.shape[0], -1, 128)
        x, _ = self.gru(x)
        x = self.fc(x[:, -1, :])
        x = self.softmax(x)
        return x

# Create the model
model = DeepSpeech2(input_shape=(128, 256, 8), num_classes=6)
print(model)

DeepSpeech2(
  (conv1): Sequential(
    (0): Conv2d(8, 128, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(128, 128, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(128, 128, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU()
  )
  (gru): GRU(128, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=6, bias=True)
  (softmax): Softmax(dim=1)
)


In [81]:
from sklearn.model_selection import KFold, GroupKFold
import torch
import torch.nn as nn
import torch.optim as optim
import gc
from tqdm import tqdm

all_oof = []
all_true = []
gkf = GroupKFold(n_splits=5)
LOAD_MODELS_FROM = None
EPOCHS = 10
criterion = nn.CrossEntropyLoss()

for i, (train_index, valid_index) in enumerate(gkf.split(train, train.target, train.patient_id)):
    train_index = train_index[:600]
    print('#'*25)
    print(f'### Fold {i+1}')
    train_gen = DataGenerator(train.iloc[train_index], shuffle=True, batch_size=32, augment=False)
    valid_gen = DataGenerator(train.iloc[valid_index], shuffle=False, batch_size=64, mode='valid')
    print(f'### train size {len(train_index)}, valid size {len(valid_index)}')
    print('#'*25)

    model = DeepSpeech2(input_shape=(128, 256, 8), num_classes=6)

    if LOAD_MODELS_FROM is None:
        optimizer = optim.Adam(model.parameters(), lr=1e-3)

        for epoch in range(EPOCHS):
            total_loss = 0
            model.train()
            for batch in tqdm(train_gen):
                inputs, targets = batch
                if inputs.shape[0] == 0:
                    break
                inputs = torch.permute(inputs,(0,3,1,2))
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            avg_loss = total_loss / len(train_gen)
            print(f'Epoch {epoch+1}/{EPOCHS}, Loss: {avg_loss:.4f}')


            model.eval()
            oof = []
            with torch.no_grad():
                total_oof_loss = 0
                for batch in valid_gen:
                    inputs, targets = batch
                    inputs = torch.permute(inputs,(0,3,1,2))
                    if inputs.shape[0] == 0:
                        break
                    outputs = model(inputs)
                    oof.append(outputs.numpy())
                    oof_loss = criterion(outputs, targets)
                    total_oof_loss += oof_loss.item()
                avg_oof_loss = total_oof_loss / len(valid_gen)
                print(f'Epoch {epoch+1}/{EPOCHS}, Validation loss: {avg_oof_loss:.4f}')

            oof = np.concatenate(oof)
            all_oof.append(oof)

            m=[(t.lower() + "_vote") for t in TARGETS]
            all_true.append(train.iloc[valid_index][m].values)

        torch.save(model.state_dict(), f'DS2_f{i}.pt')
    del model, oof
    gc.collect()

all_oof = np.concatenate(all_oof)
all_true = np.concatenate(all_true)

#########################
### Fold 1
### train size 600, valid size 3418
#########################


19it [00:13,  1.39it/s]


Epoch 1/10, Loss: 1.7426
Epoch 1/10, Validation loss: 1.6689


19it [00:13,  1.43it/s]


Epoch 2/10, Loss: 1.7253
Epoch 2/10, Validation loss: 1.6673


...

Epoch 8/10, Loss: 1.6914
Epoch 8/10, Validation loss: 1.6638


19it [00:17,  1.11it/s]


Epoch 9/10, Loss: 1.6841
Epoch 9/10, Validation loss: 1.6734


 72%|███████▏  | 13/18 [00:13<00:05,  1.05s/it]


KeyboardInterrupt: 

In [59]:
print(targets.shape)

torch.Size([7, 6])


In [37]:
torch.permute(inputs,(0,3,1,2)).shape

torch.Size([32, 8, 128, 256])